### Plotting Diff-in-Diffs

In [2]:
import pandas as pd
import numpy as np
import altair as alt


In [140]:
data_merge = pd.read_csv('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/20_analysis/big_merge.csv')
data_merge = data_merge.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
data_merge.head()

sex subprovince region  sample_population  enrolled_total  \
0  male   Abbotabad  urban                  0               0   
1  male   Abbotabad  rural                 60              55   
2  male      Attock  urban                  0               0   
3  male      Attock  rural                 64              60   
4  male      Awaran  urban                  0               0   

   rate_enrollment  year  
0              NaN  2004  
1         0.916667  2004  
2              NaN  2004  
3         0.937500  2004  
4              NaN  2004

In [141]:
# dropping NAs
data_merge = data_merge.dropna(axis = 0)


In [155]:
data_merge['region'].replace('2', 'rural', inplace = True)
data_merge['region'].replace('1', 'urban', inplace = True)

In [156]:
data_merge.loc[data_merge.year == 2006]

sex    subprovince region  sample_population  enrolled_total  \
717     male      Abbotabad  rural                307             287   
719     male         Attock  rural                369             349   
721     male         Awaran  rural                289             207   
723     male          Badin  rural                845             487   
725     male  Bahawal Nagar  rural                777             633   
...      ...            ...    ...                ...             ...   
1115  female         Thatta  rural                721             178   
1117  female      Upper Dir  rural                675             314   
1119  female         Vehari  rural                675             369   
1121  female           Zhob  rural                489             163   
1123  female         Ziarat  rural                424             257   

      rate_enrollment  year  
717          0.934853  2006  
719          0.945799  2006  
721          0.716263  2006  
723          0.576331  2006  
725          0.814672  2006  
...               ...   ...  
1115         0.246879  2006  
1117         0.465185  2006  
1119         0.546667  2006  
1121         0.333333  2006  
1123         0.606132  2006  

[230 rows x 7 columns]

In [157]:
data_post = data_merge[data_merge.year >= 2008]
data_pre = data_merge[data_merge.year < 2007]

In [158]:
data_pre.year.value_counts()

2004    246
2006    230
2005    218
Name: year, dtype: int64

In [159]:
# Mean enrollment rates
data_merge.groupby(['sex', 'year'])['rate_enrollment'].mean().reset_index()

sex  year  rate_enrollment
0   female  2004         0.461533
1   female  2005         0.510109
2   female  2006         0.569063
3   female  2007         0.335922
4   female  2008         0.563482
5   female  2010         0.565069
6   female  2011         0.564087
7   female  2012         0.622112
8   female  2013         0.607155
9   female  2014         0.642135
10  female  2015         0.567305
11  female  2018         0.629037
12  female  2019         0.558541
13    male  2004         0.658721
14    male  2005         0.652409
15    male  2006         0.744097
16    male  2007         0.346222
17    male  2008         0.707776
18    male  2010         0.709817
19    male  2011         0.697150
20    male  2012         0.768860
21    male  2013         0.744438
22    male  2014         0.777340
23    male  2015         0.694734
24    male  2018         0.747108
25    male  2019         0.675691

In [8]:
## Nick's code for confidence bands 
def get_reg_fit(data, yvar, xvar, alpha=0.05,col="blue"):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color=col).encode(
        x=alt.X(xvar, axis=alt.Axis(title='Years from Policy Change')),
        y=alt.X(yvar, axis=alt.Axis(title='')))
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color=col)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [64]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [100]:
# Enrolment by Gender

legend = alt.Chart(data_merge).transform_calculate(f= "'female'", m= "'male'")
scale = alt.Scale(domain=["Female", "Male"], range=['red', 'blue'])

## Female Trends
alt.data_transformers.disable_max_rows()
before = alt.Chart(
    data_merge[data_merge["sex"] == 'female'], title="Enrollment Trends Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('female:N', scale=scale, title=''))

base_female = before.transform_regression("year", "rate_enrollment").mark_line()

fit, female_pre_line = get_reg_fit(
    data_pre[data_pre["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="red"
)

fit, female_post_line = get_reg_fit(
    data_post[data_post["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="red"
)



## Male Trends
alt.data_transformers.disable_max_rows()
before = alt.Chart(
    data_merge[data_merge["sex"] == 'male'], title="Enrollment Trends Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('male:N', scale=scale, title=''))

base_male = before.transform_regression("year", "rate_enrollment").mark_line()

fit, male_pre_line = get_reg_fit(
    data_pre[data_pre["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="blue"
)

fit, male_post_line = get_reg_fit(
    data_post[data_post["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="blue"
)

plots= base_female + female_pre_line + female_post_line + base_male + male_pre_line + male_post_line + sep_line
plots

alt.LayerChart(...)

### Rural Data Diff in Diff

In [160]:
# Rural Areas
rural = data_merge.loc[(data_merge.region == 'rural')]

In [161]:
rural_post = rural[rural.year >= 2008]
rural_pre = rural[rural.year <= 2006]

In [162]:
rural_pre.year.value_counts()

2006    202
2004    194
2005    162
Name: year, dtype: int64

In [163]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [164]:
# Enrolment by Gender

legend = alt.Chart(data_merge).transform_calculate(f= "'female'", m= "'male'")
scale = alt.Scale(domain=["Female", "Male"], range=['palevioletred', 'cornflowerblue'])

## Female Trends
alt.data_transformers.disable_max_rows()
before_rural = alt.Chart(
    rural[rural["sex"] == 'female'], title="Enrollment Trends in Rural Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('female:N', scale=scale, title=''))

base_female_rural= before_rural.transform_regression("year", "rate_enrollment").mark_line()

fit, female_pre_line_rural = get_reg_fit(
    rural_pre[rural_pre["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)

fit, female_post_line_rural = get_reg_fit(
    rural_post[rural_post["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)



## Male Trends
alt.data_transformers.disable_max_rows()
after_rural = alt.Chart(
    rural[rural["sex"] == 'male'], title="Enrollment Trends in Rural Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('male:N', scale=scale, title=''))

base_male_rural = after_rural.transform_regression("year", "rate_enrollment").mark_line()

fit, male_pre_line_rural = get_reg_fit(
    rural_pre[rural_pre["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

fit, male_post_line_rural = get_reg_fit(
    rural_post[rural_post["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

plots_rural = base_female_rural + female_pre_line_rural + female_post_line_rural + base_male_rural + male_pre_line_rural + male_post_line_rural + sep_line
plots_rural

alt.LayerChart(...)

### Urban Areas Diff in Diff

In [165]:
# Urban Areas
urban = data_merge.loc[data_merge.region == 'urban']

In [166]:
urban_post = urban[urban.year >= 2008]
urban_pre = urban[urban.year <= 2006]

In [167]:
urban_pre.year.value_counts()

2005    56
2004    52
2006    28
Name: year, dtype: int64

In [168]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line_urban = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [169]:
# Enrolment by Gender

legend = alt.Chart(data_merge).transform_calculate(f= "'female'", m= "'male'")
scale = alt.Scale(domain=["Female", "Male"], range=['palevioletred', 'cornflowerblue'])

## Female Trends
alt.data_transformers.disable_max_rows()
before_urban = alt.Chart(
    urban[urban["sex"] == 'female'], title="Enrollment Trends in Urban Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('female:N', scale=scale, title=''))

base_female_urban= before_urban.transform_regression("year", "rate_enrollment").mark_line()

fit, female_pre_line_urban = get_reg_fit(
    urban_pre[urban_pre["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)

fit, female_post_line_urban = get_reg_fit(
    urban_post[urban_post["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)



## Male Trends
alt.data_transformers.disable_max_rows()
after_urban = alt.Chart(
    urban[urban["sex"] == 'male'], title="Enrollment Trends in Urban Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('male:N', scale=scale, title=''))

base_male_urban = after_rural.transform_regression("year", "rate_enrollment").mark_line()

fit, male_pre_line_urban = get_reg_fit(
    urban_pre[urban_pre["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

fit, male_post_line_urban = get_reg_fit(
    urban_post[urban_post["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

plots_urban = base_female_urban + female_pre_line_urban + female_post_line_urban + base_male_urban + male_pre_line_urban + male_post_line_urban + sep_line_urban
plots_urban

alt.LayerChart(...)

## Women diff-in-diff for taliban controlled cities vs not

In [98]:
women = data_merge.loc[data_merge.sex == 'female']
women.region.value_counts()

rural    1178
urban    1009
2         101
1          14
Name: region, dtype: int64